In [ ]:
using DCAUtils, HopfieldDCA, Tullio, BenchmarkTools, Zygote, LinearAlgebra
Z = read_fasta_alignment("../DataAttentionDCA\\data\\PF00014\\PF00014_mgap6.fasta.gz",0); W, M_eff = DCAUtils.compute_weights(Z,0.2);@tullio delta_j[a, j, m] := a .== Z[j, m] (a in 1:20); @tullio delta_i[a, i, m] := a .== Z[i, m] (a in 1:20);N = size(Z)[1]; M = size(Z)[2]; q = 20;  H = 10; lambdaK=5;lambdaV=5;delta_la = Matrix(I,q,q) ; alg_var = HopPlmVar(N,M_eff,q,H,lambdaK, lambdaV, Z,W, delta_i, delta_j, delta_la);K = rand(N,N,H); V = rand(q, H); g_K = rand(N*N*H); g_V = rand(q*H);


In [ ]:
a1,b1 = gradient((p1,p2)->get_loss_and_grad_zyg(p1, p2, alg_var),K,V); a,b,l = get_loss_and_grad(K, V, alg_var); println(sum(a .- a1)); println(sum(b.-b1));
#works well on gradient on k, not on be, maybe add regularization and the stretching of the parameters

In [ ]:
using HopfieldDCA, BenchmarkTools, DCAUtils, PyPlot, Zygote
filepath = "../DataAttentionDCA\\data\\PF00014\\PF00014_mgap6.fasta.gz"; M = 1000;
lambdaK = 0.5; lambdaV = lambdaK;  H = 2; alg_var = define_var(filepath,H,0.5,0.5);#alg_var = define_var(filepath,H,0.5,0.5,M);
@time grad = check_with_zyg(alg_var);
close("all"); scatter(grad[1], grad[2]); gcf();

l_r = 0.005; n_epoch = 100; print_rate = 5;trainer_small(n_epoch, l_r, l_r, alg_var, print_rate, false);


In [ ]:
using HopfieldDCA#, BenchmarkTools#, DCAUtils, PyPlot, Zygote, Tullio
using DCAUtils
filepath = "../DataAttentionDCA\\data\\PF00014\\PF00014_mgap6.fasta.gz"; Z = read_fasta_alignment(filepath,0.99);#[:,1:100];
lambdaK = 0.005; lambdaV = lambdaK;  H = 2; alg_var = HopPlmVar(H,lambdaK,lambdaV, Z);
#lambdaK = 0.005; lambdaV = lambdaK;  H = 2; alg_var = define_var(filepath,H,lambdaK,lambdaV);
K = rand(alg_var.N, alg_var.N, alg_var.H); V = rand(alg_var.q, alg_var.H); tmp = StgArr(alg_var);
en = rand(alg_var.q, alg_var.N, length(alg_var.W)); data_en = rand(alg_var.N, length(alg_var.W));
loss = rand(alg_var.N) ; reg_k = rand(alg_var.N);
@benchmark lz = get_loss_and_grad_zyg(K, V, alg_var)
@benchmark lz2 = get_loss_and_grad_zyg2(K, V, alg_var, tmp)
@benchmark lz3 = get_loss_and_grad_zyg3($K, $V, $alg_var, $en, $data_en, $loss, $reg_k)
@time az,bz = gradient((p1,p2)->get_loss_and_grad_zyg(p1, p2, alg_var),K,V);
@time az2,bz2 = gradient((p1,p2)->get_loss_and_grad_zyg2(p1, p2, alg_var, tmp),K,V);
println([sum(az .-az2),sum(bz .-bz2),lz-lz2])

K = rand(alg_var.N, alg_var.N, alg_var.H); V = rand(alg_var.q, alg_var.H); tmp = StgArr(alg_var);
en = rand(alg_var.q, alg_var.N, length(alg_var.W)); data_en = rand(alg_var.N, length(alg_var.W));
loss = rand(alg_var.N) ; reg_k = rand(alg_var.N);
@time a,b,l = get_loss_and_grad(K, V, alg_var);
@time a2,b2,l2 = get_loss_and_grad2(K, V, alg_var, tmp);
@time az,bz = gradient((p1,p2)->get_loss_and_grad_zyg(p1, p2, alg_var),K,V);
println([sum(a .-a2), sum(b .-b2), l-l2])
println([sum(a .-az), sum(b .-bz)])

using HopfieldDCA; import DCAUtils: read_fasta_alignment;#, BenchmarkTools#, DCAUtils, PyPlot, Zygote, Tullio
filepath = "../DataAttentionDCA\\data\\PF00014\\PF00014_mgap6.fasta.gz"; Z = read_fasta_alignment(filepath,0.99);#[:,1:100];
lambdaK = 0.005; lambdaV = lambdaK;  H = 2; alg_var = HopPlmVar(H,lambdaK,lambdaV, Z);
#lambdaK = 0.005; lambdaV = lambdaK;  H = 2; alg_var = define_var(filepath,H,lambdaK,lambdaV);
K = rand(alg_var.N, alg_var.N, alg_var.H); V = rand(alg_var.q, alg_var.H); tmp = StgArr(alg_var);
@time a,b,l = get_loss_and_grad(K, V, alg_var);


@benchmark lz = [get_loss_and_grad_zyg(K, V, alg_var) for i in 1:10]
@benchmark lz2 = [get_loss_and_grad_zyg2(K, V, alg_var, tmp) for i in 1:10]
@benchmark lz3 = [get_loss_and_grad_zyg3(K, V, alg_var, en, data_en, loss, reg_k) for i in 1:10]


In [ ]:
using HopfieldDCA
filepath = "../DataAttentionDCA\\data\\PF00014\\PF00014_mgap6.fasta.gz"; lambdaK = 0.005; lambdaV = lambdaK;  
H = 1; alg_var = define_var(filepath,H,lambdaK,lambdaV); m = trainer(alg_var, 5);

In [ ]:
using HopfieldDCA
filepath = "../DataAttentionDCA\\data\\PF00014\\PF00014_mgap6.fasta.gz"; 

for h in [2, 10, 30, 50, 70]
    for lambda in [10^-5, 10^-3, 10^-2, 10^-1]

        lambdaK = lambda; lambdaV = lambdaK; alg_var = define_var(filepath,h,lambdaK,lambdaV);
        @time m = trainer(alg_var, 100);
    end
end